# Amazon SageMaker + WhyLabs

This example shows how to deploy a SageMaker endpoint with WhyLabs integration.

In [ ]:
!pip install boto3==1.18.39 python-dotenv==0.19.0 scikit-learn==0.24.2 pandas==1.3.2

In [1]:
import os
import time
import json
import random
import urllib.request as urllib
from joblib import dump
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import boto3
from dotenv import dotenv_values
from utils import delete_model, delete_endpoint_config, delete_endpoint, is_endpoint_running

## Table of Contents
- [1 - AWS Configuration](#1)
- [2 - Train a Model](#2)
- [3 - Custom image building and pushing to ECR](#3)
- [4 - Create SageMaker Endpoint](#4)
    - [a. Model Creation](#a)
    - [b. Endpoint Config Creation](#b)
    - [c. Endpoint Creation](#c)
- [5 - Test Endpoint](#5)
- [6 - Delete AWS resources](#6)

<a name='1'></a>
## 1 - AWS configuration

In [2]:
AWS_PROFILE_NAME = "default"
session = boto3.session.Session(profile_name=AWS_PROFILE_NAME)
AWS_REGION_NAME = session.region_name

In [3]:
sts = session.client("sts")
sm = session.client('sagemaker', region_name=AWS_REGION_NAME)
AWS_ACCOUNT_ID = sts.get_caller_identity().get("Account")
DOCKER_IMAGE_NAME = "whylabs-sagemaker"

<a name='2'></a>
## 2 - Train a Model

Download Iris Species dataset:

In [4]:
# Download Iris dataset and save it as csv
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
raw_data = urllib.urlopen(url)
try:
    os.mkdir("code/dataset/")
    # Save data as csv
    with open('code/dataset/Iris.csv', 'wb') as file:
        file.write(raw_data.read())
        print("Dataset downloaded successfully!")    
except Exception as e:
    print(" 'dataset' directory already existed. Moving forward")

 'dataset' directory already existed. Moving forward


Split data set into train and test sets

In [5]:
# Load data
data = pd.read_csv('code/dataset/Iris.csv', header=None)
# Separating the independent variables from dependent variables
X = data.iloc[:, 0:4].values
y = data.iloc[:, -1].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

Train the SVM classifier

In [6]:
# Train a classifier
print("Train started.")
model = SVC()
model.fit(x_train, y_train)
print("Train finished.")
# Save the model
dump(model, 'code/model.joblib')
print("Model saved as model.joblib")

Train started.
Train finished.
Model saved as model.joblib


Split data set into train and test sets

<a name='3'></a>
## 3 - Custom image building and pushing to ECR

In [7]:
os.system(f"./build_push.sh {DOCKER_IMAGE_NAME} {AWS_PROFILE_NAME}")

Image name whylabs-sagemaker
Profile name default
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:377983720232:repository/whylabs-sagemaker",
            "registryId": "377983720232",
            "repositoryName": "whylabs-sagemaker",
            "repositoryUri": "377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker",
            "createdAt": "2021-09-08T17:32:14-05:00",
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}
Login Succeeded


#1 [internal] load build definition from Dockerfile
#1 sha256:ae478a8a0c822118d386950ba7913cd06eef785451d66abe7b01b5fbd6cd6285
#1 transferring dockerfile: 1.50kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:b54083463265b41c72a5dee01c386777b758184f41d547f4781fcb7c074a98f3
#2 transferring context: 171B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:98682aacae76ab9198aede26bb6cd7176e2e4b329b793163c199e86e47b639b9
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 1.9s

#5 [ 1/11] FROM docker.io/library/ubuntu:18.04@sha256:9bc830af2bef73276515a29aa896eedfa7bdf4bdbc5c1063b4c457a4bbb8cd79
#5 sha256:d1750e31869fe5a60e2fad31896f5d8b06a6c26d3a20b7f5836401e641279689
#5 DONE 0.0s

#8 [in

The push refers to repository [377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker]
5f70bf18a086: Preparing
5fd0815f8e1d: Preparing
0db6bc60cf67: Preparing
66d55ab066d7: Preparing
94df9b81d20b: Preparing
1d32553c20b9: Preparing
87953760e835: Preparing
42fafac3315e: Preparing
d20859943999: Preparing
cb8615e15e41: Preparing
6babb56be259: Preparing
42fafac3315e: Waiting
d20859943999: Waiting
cb8615e15e41: Waiting
6babb56be259: Waiting
1d32553c20b9: Waiting
87953760e835: Waiting
0db6bc60cf67: Layer already exists
5f70bf18a086: Layer already exists
66d55ab066d7: Layer already exists
94df9b81d20b: Layer already exists
87953760e835: Layer already exists
d20859943999: Layer already exists
1d32553c20b9: Layer already exists
42fafac3315e: Layer already exists
6babb56be259: Layer already exists
cb8615e15e41: Layer already exists
5fd0815f8e1d: Pushed
latest: digest: sha256:f6b24a578d3cb55c4269545a878b7efe2c28e63de58fbfebc3b290d88ca55064 size: 2615


0

<a name='4'></a>
## 4 - Create SageMaker Endpoint

The steps to deploy a SageMaker model are:

1. Create a model
2. Create an endpoint configuration
3. Create a SageMaker endpoint

<a name='a'></a>
### a. Model Creation

**Replace the following Role ARN accordingly.**

In [ ]:
EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecution"

In [ ]:
ECR_IMAGE_URI = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION_NAME}.amazonaws.com/{DOCKER_IMAGE_NAME}:latest"
ENDPOINT_NAME = "whylabs-sagemaker"
INSTANCE_TYPE = "ml.m4.xlarge"

Load variables important for __WhyLabs configuration__ defined inside __.env file__ as dictionary. This values will be settled once the docker container is running within SageMaker.

In [ ]:
# Load .env file as dictionary
environment = dotenv_values("code/.env")

In [ ]:
# ECR image to be used
PRIMARY_CONTAINER = {
    'Image': ECR_IMAGE_URI, 
    "Environment": environment,
}

In [ ]:
try:
    # Create sagemaker model
    r = sm.create_model(
        ModelName=ENDPOINT_NAME,
        ExecutionRoleArn=EXECUTION_ROLE_ARN,
        PrimaryContainer=PRIMARY_CONTAINER,
    )
    print("SageMaker model created.")
except Exception as e:
    print(e.response["Error"])

<a name='b'></a>
### b. Endpoint Config creation

In [ ]:
ENDPOINT_CONFIG_NAME = ENDPOINT_NAME + '-config'

In [ ]:
try:
    # create endpoint configuration
    _ = sm.create_endpoint_config(
        EndpointConfigName=ENDPOINT_CONFIG_NAME,
        ProductionVariants=[
            {
                'InstanceType': INSTANCE_TYPE,
                'InitialVariantWeight': 1,
                'InitialInstanceCount': 1,
                'ModelName': ENDPOINT_NAME,
                'VariantName': 'AllTraffic'
            }
        ]
    )
    print("Endpoint configuration created.")
except Exception as e:
    print(e.response["Error"])

<a name='c'></a>
### c. Endpoint creation

In [ ]:
try:
    # create endpoint
    r = sm.create_endpoint(
        EndpointName=ENDPOINT_NAME,
        EndpointConfigName=ENDPOINT_CONFIG_NAME
    )
    print(f"Completed {ENDPOINT_NAME} model endpoint deployment !!!")
except Exception as e:
    print(e.response["Error"])

<a name='5'></a>
## 5 - Test Endpoint 

You have to wait to the model to be in "InService" status to test it.

In [ ]:
def modify_random_column_values(data, value: float = np.random.uniform(low=0.0, high=10.0)) -> None:
    random_column = None
    data_mod = data.copy(deep=True)
    try:
        number_of_columns = len(data_mod.columns) - 2 # Index and label eliminated
        random_column = data_mod.columns[np.random.randint(number_of_columns) + 1]
        data_mod[random_column] = value
    except Exception as ex:
        raise f"Error adding fix value in random column: {str(random_column)}"
    return data_mod
        
        
def add_random_column_outliers(data, number_outliers: int = 10) -> None:
    random_column = None
    data_mod = data.copy(deep=True)
    try:
        number_of_columns = len(data_mod.columns) - 2  # Index and label eliminated
        number_of_rows = data_mod.shape[0]
        random_column = data_mod.columns[np.random.randint(number_of_columns) + 1]
        for i in range(number_outliers):
            random_row = np.random.randint(0, number_of_rows)
            data_mod.loc[random_row, random_column] = round(np.random.uniform(low=20.0, high=50.0), 2)
    except Exception as ex:
        raise f"Error adding outliers in random column: {random_column}"
    return data_mod

In [ ]:
labels = ["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"]

In [ ]:
# modify a variable distribution
data_mod = add_random_column_outliers(data, 30)
print("Dataset distribution modified!")

In [ ]:
# Invoke the endpoint using
sg = session.client("runtime.sagemaker", region_name=AWS_REGION_NAME)
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)
# Check if model was created successfully
if status == "InService":
    while True:
        # Build a payload with random values
        payload = dict(zip(labels, random.choice(data_mod.iloc[:, 0:4].values)))
        payload = json.dumps(payload)
        # Send payload to sagemaker endpoint
        response = sg.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            Body=payload,
            ContentType='application/json',
        )
        # Decode the response
        print(json.loads(response["Body"].read().decode("utf-8")))
        time.sleep(5)
else:
    print(f"Endpoint status is {status}.")

Response should look like this:
```bash
{'data': {'class': 'Iris-setosa'}, 'message': 'Success'}
```

<a name='6'></a>
## 6 - Delete AWS resources

In [ ]:
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)
status

In [ ]:
if status in ["InService", "Failed"]:
    delete_model(sm, ENDPOINT_NAME)
    delete_endpoint_config(sm, ENDPOINT_CONFIG_NAME)
    delete_endpoint(sm, ENDPOINT_NAME)